In [2]:
def getInput():
    return open("../input/day_5.txt").read()

def getInputLines():
    return getInput().split("\n")

# Day 5 Part 1

parsing: /`(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)`/gms

In [3]:
import re

text = getInput()

# text = """
# seeds: 79 14 55 13

# seed-to-soil map:
# 50 98 2
# 52 50 48

# soil-to-fertilizer map:
# 0 15 37
# 37 52 2
# 39 0 15

# fertilizer-to-water map:
# 49 53 8
# 0 11 42
# 42 0 7
# 57 7 4

# water-to-light map:
# 88 18 7
# 18 25 70

# light-to-temperature map:
# 45 77 23
# 81 45 19
# 68 64 13

# temperature-to-humidity map:
# 0 69 1
# 1 0 69

# humidity-to-location map:
# 60 56 37
# 56 93 4
# """.strip()

# match all numbers, first match is seed line, subsequent matches are the maps
seeds, *maps = [
    [int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)]
    for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)
    ]
# sort ranges in maps based on their start number
maps = [sorted(r, key=lambda x: x[1]) for r in
    [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]]

# main
locs = []
for seed in seeds:
    temp = seed
    for map in maps:
        for (dst, src, span) in map:
            if src <= temp < src + span:
                temp = dst + (temp - src)
                break
    locs.append(temp)
min(locs)
# 331445006


331445006

# Day 5 Part 2

# 24092691 too high
# 20522066 too high
# 19624706 too high

In [150]:
import re

text = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

# text = getInput()

seeds, *maps = [[int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)] for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)]
seeds = [seeds[i:i+2] for i in range(0, len(seeds), 2)]
maps = [sorted(r, key=lambda x: x[1]) for r in [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]]

# I'm gonna have to write real code for this one...

def extract_range(seed_range, map_range):
    processed, passed_through = [], []
    
    seed_start, seed_length = seed_range
    seed_end = seed_start + seed_length - 1
    map_dst, map_start, map_length = map_range
    map_end = map_start + map_length - 1

    # no intersection
    if (seed_start > map_end) or (map_start > seed_end):
        passed_through.append([seed_start, seed_length])
    else:
        """
        #: map, *: seed

        case 0
        ##############
            *****
            ^^^^^
        case 1
            #####    
        **************
            ^^^^^
        case 2
        #########
            **********
            ^^^^^
        case 3
            ##########
        *********
            ^^^^^
        """

        overlap_start = max(map_start, seed_start)
        overlap_end = min(map_end, seed_end)
        map_shift = map_dst - map_start
        processed.append([overlap_start + map_shift, overlap_end - overlap_start + 1])

        if overlap_start > seed_start:
            passed_through.append([seed_start, overlap_start - seed_start])
        if overlap_end < seed_end:
            passed_through.append([overlap_end + 1, seed_end - overlap_end])

    return [processed, passed_through]



seed_ranges = [[s,l] for s,l in seeds]
for map_ranges in maps:
    for sr in seed_ranges:

        for mr in map_ranges:


        
print(min(seed_ranges, key=lambda x:x[0])[0])

        


KeyboardInterrupt: 

In [139]:
(extract_range((10,10),(100,20,10)), # no overlap
extract_range((10,10),(100,11,8)), # internal overlap
extract_range((10,10),(100,0,20)), # external overlap
extract_range((10,10),(100,5,10)), # left overlap
extract_range((10,10),(100,15,10)),) # right overlap

([[10, 10]],
 [[100, 8], [10, 2], [19, 1]],
 [[110, 10]],
 [[105, 5], [15, 5]],
 [[100, 5], [10, 6]])